In [26]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

In [2]:
lines = pd.read_csv('./datasets/raw/cornell-movie-dialogs-corpus/movie_lines.txt',
                    sep=re.escape(' +++$+++ '),
                    names=['lineID', 'characterID', 'movieID', 'characterName', 'utterance'],
                    index_col=0,
                    engine='python', 
                    encoding="latin1")

conversations = pd.read_csv('./datasets/raw/cornell-movie-dialogs-corpus/movie_conversations.txt',
                    sep=re.escape(' +++$+++ '),
                    names=['characterID_0', 'characterID_1', 'movieID', 'utteranceList'],
                    engine='python', 
                    encoding="latin1")

In [35]:
def generate_dataset(conversations, lines):
    dataset = []
    for index, row in tqdm(conversations.iterrows(), total=conversations.shape[0]):
        prompt = ''
        line_ids = eval(row['utteranceList'])
        for line_id in line_ids[:-1]:
            line = lines.loc[line_id]
            prompt += f'{str(line.characterName).title()}: {re.sub(" +", " ", str(line.utterance))}\n'
        last_line = lines.loc[line_ids[-1]]
        prompt += f'{str(last_line.characterName).title()}:'
        completion = f' {re.sub(" +", " ", str(last_line.utterance))}'
        dataset.append({'prompt': prompt, 'completion': completion})
    return pd.DataFrame(dataset)

dataset = generate_dataset(conversations, lines)
dataset.head()

  0%|          | 0/83097 [00:00<?, ?it/s]

,prompt,completion
0,Bianca: Can we make this quick? Roxanne Korrin...,Okay... then how 'bout we try out some French...
1,Bianca: You're asking me out. That's so cute. ...,Forget it.
2,"Bianca: No, no, it's my fault -- we didn't hav...",Seems like she could get a date easy enough...
3,Cameron: Why?\nBianca: Unsolved mystery. She u...,That's a shame.
4,"Bianca: Gosh, if only we could find Kat a boyf...",Let me see what I can do.


In [47]:
dataset.iloc[np.random.randint(dataset.shape[0])].to_dict()

{'prompt': 'Margo: How fortunate that I have an understudy so ready, so willing and so able to go on.\nLloyd: The audience will want its money refunded, believe me.\nMargo:',
 'completion': ' Thank you, Lloyd. Godspeed.'}

In [44]:
dataset.to_csv('datasets/dialogs_dataset.csv', index=False)